# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [8]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
from HouseholdSpecializationModel import HouseholdSpecializationModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [3]:
# code for solving the model (remember documentation and comments)

from HouseholdSpecializationModel import HouseholdSpecializationModelClass

# Laver nu interaktiv figur
# a. import
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets

# b. plotting figure
def interactive_figure(X, sigma_guess, alpha_guess):
    
    # i. normal distribution from guess
    F = norm(loc=alpha_guess,scale=sigma_guess)
    
    # ii. x-values
    x_low = F.ppf(0.001)
    x_high = F.ppf(0.999)
    x = np.linspace(x_low,x_high,100)

    # iii. figure
    fig = plt.figure(dpi=100)
    ax = fig.add_subplot(1,1,1)
    ax.plot(x,F.pdf(x),lw=2)
    ax.hist(X,bins=100,density=True,histtype='stepfilled');
    ax.set_ylim([0,0.5])
    ax.set_xlim([-6,6])

# c. parameters
alpha_true = 0.5
sigma_true = 1
alpha_guess = 1
sigma_guess = 0.5

# d. random draws
X = np.random.normal(loc=alpha_true,scale=sigma_true,size=10**6)

# e. interactive
widgets.interact(interactive_figure,
    X=widgets.fixed(X),
    alpha_guess=widgets.FloatSlider(description="alpha_guess", min=0.25, max=0.75, step=0.25, value=1), 
    sigma_guess=widgets.FloatSlider(description="sigma_guess", min=0.5, max=1.5, step=0.25, value=1), 
);

interactive(children=(FloatSlider(value=1.0, description='sigma_guess', max=1.5, min=0.5, step=0.25), FloatSli…

In [7]:
# code for solving the model (remember documentation and comments)

from HouseholdSpecializationModel import HouseholdSpecializationModelClass

# Vi laver nu den endelige figur
alphas = np.random.uniform(low=0.25, high=0.75)
sigmas = np.random.uniform(low=0.50, high=1.50)

def interactive_figure(alphas,sigmas,HF,HM):

    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(H)

widgets.interact(interactive_figure,
    alphas=widgets.FloatSlider(description="alpha", min=0.25, max=0.75, step=0.25, value=2),
    sigmas=widgets.FloatSlider(description="sigma", min=0.50, max=1.5, step=0.50, value=2),
);

ValueError: cannot find widget or abbreviation for argument: 'HF'

# Question 2

Explain your code and procedure

In [3]:
# code

# Question 3

Explain your code and procedure

In [4]:
# code

ADD CONCISE CONLUSION.